In [ ]:
import pandas as pd
import datetime
import re
import time

In [ ]:
# df1 = pd.read_csv('CSL.AX001.csv')
# df2 = pd.read_csv('CSL.AX002.csv')

In [ ]:
df1 = pd.read_excel(r'New folder\BHP.AX001.xlsx')
df2 = pd.read_excel(r'New folder\BHP.AX002.xlsx')

Fill the empty Data Column with Previous Cell Value

In [ ]:
df1.Date = df1.Date.fillna(method='ffill')

In [ ]:
df1 = df1.fillna('')

Converting into Datetime Object

In [ ]:
# time.sleep(5)
df1.Date = pd.to_datetime(df1.Date)

In [ ]:
df2.Year = pd.to_datetime(df2.Year, format=r'%Y-%m-%d')

In [ ]:
def remove_str(row):
    x = row.Comments2
    try:
        x = x.replace(re.search('Originally posted .*', x)[0], '').strip()
    except:
        pass
    try:
        x = x.replace(re.search('Release Date:.*', x)[0], '').strip()
    except:
        pass
    row.Comments2 = x
    return row
df1 = df1.apply(remove_str, axis='columns')

Previous Year's Number of Post

In [ ]:
def prev_year(date_temp):
    return df1[(((df1.Date - date_temp).astype('timedelta64[h]')/24) >= -365) & (((df1.Date - date_temp).astype('timedelta64[h]')/24) <=0)].shape[0]

90 days post number

In [ ]:
def num_of_post(filtered_df):
    date_90_len = filtered_df.shape[0]
    return date_90_len

Remove Unwanted Lines from Comments

In [ ]:
# filtered_df = filtered_df.apply(remove_str, axis='columns')

Row containing at least one negative words

In [ ]:
def row_containing_neg_words(filtered_df, keyword):
    def func_inner(row):      
        for each_word in keyword:
            if each_word in str(row.Comments2):
                return 1
        return 0
    return sum(filtered_df.apply(func_inner, axis='columns'))

Number of Negative Words Mentioned

In [ ]:
def criticism_words(filtered_df, keyword):
    def func2(row):
        word_count = 0
        for each in keyword:
            word_count += len(re.findall(each.lower(), str(row.Comments2)))
        return word_count
    return sum(filtered_df.apply(func2, axis='columns'))

Total Number of Words

In [ ]:
def words_length(filtered_df):
    def func3(row):
        return len(str(row.Comments2).split())
    return sum(filtered_df.apply(func3, axis='columns'))

Output

In [ ]:
df2.Year

In [ ]:
def main_function(row): 
    keyword = ['am', 'is', 'are', 'was', 'were']
    row['Total Post in a Financial Year'] = prev_year(row['Year'])
    filtered_df = df1[(abs((df1.Date - row['Year']).astype('timedelta64[h]')/24) <= 90)]
    # print(filtered_df.isna().any())
    row['Total post related to earnings'] = num_of_post(filtered_df)
    row['Criticism Comments'] = row_containing_neg_words(filtered_df, keyword)
    row['Criticism Words'] = criticism_words(filtered_df, keyword)
    row['Total no. of words'] = words_length(filtered_df)
    return row
df2 = df2.apply(main_function, axis='columns')

In [ ]:
df2.to_csv('3.csv')

In [ ]:
def main_function_temp(dat): 
    keyword = ['am', 'is', 'are', 'was', 'were']
    a = prev_year(dat)
    filtered_df = df1[(abs((df1.Date - dat).astype('timedelta64[h]')/24) <= 90)]
    # print(filtered_df.isna().any())
    b = num_of_post(filtered_df)
    c = row_containing_neg_words(filtered_df, keyword)
    d = criticism_words(filtered_df, keyword)
    e = words_length(filtered_df)
    return a, b, c, d, e

In [ ]:
for i in range(0, df2.shape[1]):
    print(main_function_temp(df2.Year[i]))
    a, b, c, d, e = main_function_temp(df2.Year[i])
    df2['Total Post in a Financial Year'][i] = a
    df2['Total post related to earnings'] = b
    df2['Criticism Comments'] = c
    df2['Criticism Words'] = d
    df2['Total no. of words'] = e

In [ ]:
# df2.Year = pd.to_datetime(df2['Year'])

In [ ]:
filtered_df = df1[(abs((df1.Date - df2['Year'][5]).astype('timedelta64[h]')/24) <= 90)]


In [ ]:
filtered_df

In [ ]:
words_length(filtered_df)

In [ ]:
df2.Year